# Calculs asynchrones

Dans cette séance, vous utiliserez le module `concurrent.futures` qui fournit une API simple pour paralléliser des calculs même dans le cas où le cadre Map-Reduce ne s'applique pas.

## `Executor.submit`

La méthode `submit` lance un calcul dans un nouveau _thread_ (ou _process_, selon le type d'`Executor` choisi). Cette méthode retour un objet `Future` qui recevra le résultat. Pour collecter ce résultat, on fait appel à la méthode `.result()` de cet objet `Future`, qui retournera le résultat, une fois disponible.

**Question.** Implémentez une fonction `slowadd` qui prend en entrée deux nombres et en retourne la somme, après avoir attendu une seconde. Utilisez cette fonction pour calculer la somme $(3+7)$ et vérifiez que le calcul prend environ une seconde à s'exécuter.

In [3]:
%%time
from time import sleep

def slowadd(a, b, delay=1):
    sleep(delay)
    return a + b

slowadd(3, 7)

CPU times: user 836 µs, sys: 1.38 ms, total: 2.22 ms
Wall time: 1 s


10

**Question.** Répétez l'opération en encapsulant votre appel à la fonction `slowadd` dans un [`ThreadPoolExecutor.submit`](https://docs.python.org/3/library/concurrent.futures.html#concurrent.futures.Executor.submit). Combien de temps prendra le calcul cette fois-ci ? Changer le paramètre `max_workers` aura-t-il un effet ici ? Vérifiez.

In [4]:
%%time
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor(max_workers=1) as e:
    future = e.submit(slowadd, 3, 7)
    print(future.result())


10
CPU times: user 1.88 ms, sys: 1.53 ms, total: 3.41 ms
Wall time: 1 s


**Question.** Supposons que l'on dispose d'une liste `li = [1, 7, 9, 2]` et que l'on souhaite utiliser `slowadd` pour calculer $(3+x)$ où $x$ prend successivement chacune des valeurs de la liste `li`. Combien de temps prendra ce calcul si l'on ne met pas en place de stratégie de calcul parallèle ? Vérifiez.

In [5]:
%%time
li = [1, 7, 9, 2]

print([slowadd(3, x) for x in li])

[4, 10, 12, 5]
CPU times: user 1.26 ms, sys: 1.56 ms, total: 2.83 ms
Wall time: 4.01 s


**Question.** Reprenez maintenant le code ci-dessus utilisant un `ThreadPoolExecutor` pour permettre l'exécution des calculs de la question précédente en parallèle. À quelle valeur minimale faut-il fixer `max_workers` ? Combien de temps prendra le calcul cette fois ? Vérifiez.

In [6]:
%%time
with ThreadPoolExecutor(max_workers=len(li)) as e:
    futures = [e.submit(slowadd, 3, x) for x in li]
    results = [f.result() for f in futures]
    print(results)

[4, 10, 12, 5]
CPU times: user 2.67 ms, sys: 2.8 ms, total: 5.47 ms
Wall time: 1.01 s


## Exercice de synthèse

Dans cet exercice, on suppose qu'on a à notre disposition un ensemble de vecteurs $(x_0, \dots, x_{n-1})$ de $\mathbb{R}^p$ et que l'on veut calculer leurs distances deux à deux et stocker ces distances dans une matrice $A$ de taille $n\times n$ telle que :

$$A_{i, j} = d(x_i, x_j)^2$$

On prendra pour distance $d(\cdot, \cdot)$ la distance Euclidienne, et l'on utilisera le fait que pour cette distance, on a :

$$d(a, b)^2 = \|a\|^2 + \|b\|^2 - 2 \left\langle a, b \right\rangle$$

**Question.** Calculez la matrice $A$ obtenue pour le jeu de données Olivetti Faces obtenu comme suit :

In [7]:
from sklearn.datasets import fetch_olivetti_faces

X, _ = fetch_olivetti_faces(return_X_y=True)

Pour calculer cette matrice, vous prendrez soin :
* d'utiliser l'astuce décrite ci-dessus
* de ne pas calculer plusieurs fois les termes de normes au carré

In [19]:
%%time
import numpy as np

with ThreadPoolExecutor(max_workers=5) as e:
    A = [[0. for _ in range(len(X))] for _ in range(len(X))]
    normes_au_carre = [e.submit(np.dot, x, x) for x in X]
    produits_scalaires = [[e.submit(np.dot, X[i], X[j]) for j in range(len(X))] for i in range(len(X))]
    for i in range(len(X)):
        for j in range(len(X)):
            A[i][j] = normes_au_carre[i].result() + normes_au_carre[j].result() - 2 * produits_scalaires[i][j].result()

CPU times: user 3.08 s, sys: 1.31 s, total: 4.4 s
Wall time: 3.6 s


**Question.** Comparez l'efficacité de votre implémentation avec celle de la fonction [`cdist`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html) de `scipy`.

In [16]:
%%time
from scipy.spatial.distance import cdist

A = cdist(X, X, metric="sqeuclidean")

CPU times: user 168 ms, sys: 4.54 ms, total: 173 ms
Wall time: 171 ms


## Exercice de synthèse #2

**TODO** : exercice avec des téléchargements mais pas un simple map-reduce pour illustrer un cas où c'est utile